In [19]:
#------------------------------------------------------------------------------+
#
#   Samar shaaban Haytamy
#   PSO service to select optimal web service  with Python
#   OCT, 2018
#
#------------------------------------------------------------------------------+

#--- IMPORT DEPENDENCIES ------------------------------------------------------+

from __future__ import division
import random
import time

In [2]:
from pandas import DataFrame
#from pandas import Series
#from pandas import concat
from pandas import read_csv
#from pandas import datetime
import numpy as np

In [3]:
#read future QoS_provision data
import glob
import pandas as pd
files=glob.glob("csv_predictions/*.csv")
data=[]
providers_names=[]
for file in files:
    df=read_csv(file,header=0,names=['Av','Mx_rt','Mn_rt','Avg_rt','th'])
    providers_names.append(file)
    df.drop(df.columns[[1,2]], axis=1, inplace=True)
    data.append(df)

In [4]:
services=[]
for sublist in data:
    services.append(sublist.values)
print(services[0])

[[0.8676461  0.48452288 0.47976446]
 [0.83152074 0.5907682  0.588747  ]
 [0.8421104  0.54637253 0.54049224]
 [0.7373683  0.6523395  0.6414325 ]]


In [5]:
#split 100 services into 3 groups
class_service_1=services[0:33]
class_service_2=services[33:66]
class_service_3=services[66:100]

### read end user future preferences weights for each service

In [8]:
user_weights_df=read_csv('future user_behavior.csv',header=0,names=['Av','Rt','Th'])

In [15]:
W=list(user_weights_df.values)

In [6]:
#test cases
#class_service_1=[[0.21,0.8,0.35],[0.021,0.5,0.12]]
#class_service_2=[[0.821,0.08,0.5],[0.901,0.4,0.15]]
#class_service_3=[[0.621,0.25,0.22], [0.321,0.08,0.05]]    #???
#W=[[0.8,0.6,0.6],[0.1,0.3,0.8],[0.621,0.25,0.22], [0.321,0.08,0.05]]          #???

In [7]:
#W[0][1]
len(class_service_2)

33

In [8]:
#--- COST FUNCTION ------------------------------------------------------------+
# function we are attempting to optimize (minimize)

def func1(x):
    total_plan_value=0
    T_peroids=len(x)//3      #Time periods=4=total points or dimensions(x)/#classes which are 3 as assumed
 #   print("X=",x)
    for i in range(T_peroids):
        # assume the services are composed parallel aggragate QoS attributes values
        aggre_rt=max(class_service_1[x[3*i+0]][i,0],class_service_2[x[3*i+1]][i,0],class_service_3[x[3*i+2]][i,0])
        aggre_th=min(class_service_1[x[3*i+0]][i,1],class_service_2[x[3*i+1]][i,1],class_service_3[x[3*i+2]][i,1])
        aggre_cost=class_service_1[x[3*i+0]][i,2]+class_service_2[x[3*i+1]][i,2]+class_service_3[x[3*i+2]][i,2]
        value_plan_over_period_i=aggre_rt*W[i][0]+aggre_th*W[i][1]+aggre_cost*W[i][2]
        total_plan_value+=value_plan_over_period_i

    return total_plan_value

In [9]:
class Particle:
    def __init__(self,x0):
        self.position_i=[]                   # particle position
        self.velocity_i=[]                   # particle velocity
        self.pos_best_i=[]                   # best position individual
        self.local_best_fitness_value=-1     # best  individual fitness value
        self.fitness_value=-1                # current individual fitness value

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness using optimization function: costFunction
    def evaluate(self,costFunc):
        self.fitness_value=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.fitness_value > self.local_best_fitness_value or self.local_best_fitness_value==-1:
            self.pos_best_i=self.position_i
            self.local_best_fitness_value=self.fitness_value

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.8       # constant inertia weight (how much to weigh the previous velocity)
        c1=2        # cognative constant
        c2=2        # social constant

        for i in range(0,num_dimensions):
            r1=random.randint(0,2)
            r2=random.randint(0,2)

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social
            

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]


            #want to make it integer number refers to services number (through bounds)==>update bounds
            #??? ok solved
            # adjust maximum position if necessary
            self.position_i[i]=int(self.position_i[i])
            if self.position_i[i]>bounds[i%3][1]:
                self.position_i[i]=bounds[i%3][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i%3][0]:
                self.position_i[i]=bounds[i%3][0]


In [25]:
class PSO():
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        value_best_g=-1                   # best value for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(x0))

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,value_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)

                # determine if current particle is the best (globally)
                if swarm[j].fitness_value > value_best_g or value_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    value_best_g=float(swarm[j].fitness_value)

            # cycle through swarm and update velocities and positioncostFunc
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print ('FINAL:')
        pos_best_g=np.array(pos_best_g)
        pos_best_g=pos_best_g.reshape(4,3)  #time period=4 , classes number=3
        print ('best global position:\n',pos_best_g)
        print ('\n \n best global target value:  ',value_best_g)
        for j in range(0,num_particles):
            print("\n \n position[",swarm[j].pos_best_i)
            print("local_best_fitness_value[",swarm[j].local_best_fitness_value)
           

In [23]:
import random
import time
random.choice(range(len(class_service_1)))

print(len(class_service_1)-1)

32


In [12]:
T_periods=4
class_numbers=3
x0=[]
for i in range(T_periods):
    x0.append(random.choice(range(len(class_service_1))))
    x0.append(random.choice(range(len(class_service_2))))
    x0.append(random.choice(range(len(class_service_3))))

In [13]:
len(x0)

12

In [14]:
bounds=[(0, len(class_service_1)-1),(0,len(class_service_2)-1),(0,len(class_service_3)-1)]

In [15]:
bounds

[(0, 32), (0, 32), (0, 33)]

In [28]:
PSO(func1,x0,bounds,num_particles=5,maxiter=100)

FINAL:
best global position:
 [[15 10  6]
 [ 0  7 24]
 [21  0 10]
 [ 0  0 33]]

 
 best global target value:   5.965020856760001

 
 position[ [7, 10, 0, 0, 0, 24, 21, 0, 0, 0, 0, 33]
local_best_fitness_value[ 5.965020856760001

 
 position[ [7, 10, 3, 0, 14, 33, 21, 0, 4, 0, 0, 33]
local_best_fitness_value[ 5.78743868536

 
 position[ [19, 10, 0, 0, 10, 33, 21, 0, 9, 0, 0, 33]
local_best_fitness_value[ 5.6324023489399995

 
 position[ [0, 10, 13, 0, 7, 22, 21, 0, 19, 0, 0, 33]
local_best_fitness_value[ 5.71154732852

 
 position[ [15, 20, 27, 0, 0, 0, 22, 0, 33, 0, 0, 33]
local_best_fitness_value[ 5.6582854231
